## DBCSAN Clustering
`另一种聚类方式：将所有数字替换为0，不经过分词直接聚类`
``` python
re.sub(r'\d+(\.\d+)?', '0', text)
```

In [1]:
import pandas as pd
import re
from utils.cluster import reassign_clusters, cluster, vectorize, tokenize,Cluster

# select the dataset
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
datasets = ['OpenStack']
for dataset in datasets:
    # load the dataset
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    # tokenize -> vectorize -> cluster -> reassign_clusters
    tokenized_logs = [tokenize(log) for log in logs]
    labels, cluster_nums = cluster(vectorize(tokenized_logs))
    labels, cluster_nums = reassign_clusters(labels, cluster_nums, tokenized_logs)

d:\develop\anaconda\envs\langchain\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
print('num of clusters:', cluster_nums)


# store the logs in the cluster
inputs = []
for i in range(cluster_nums):
    inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
for i, label in enumerate(labels):
    inputs[label][0] = label
    inputs[label][1].append(logs[i])
    inputs[label][2].append(i)
    if inputs[label][3] == '':
        inputs[label][3] = df['EventTemplate'][i]

# check the cluster k
k = 0
lengh_cluster = len(inputs[k][1])
print('cluster ', k)
print('length:', lengh_cluster)
print('template:', inputs[k][3])
print('-'*20)
for log in set(inputs[k][1]):
    print(log)

num of clusters: 44
cluster  0
length: 929
template: <*> "GET <*>" status: <*> len: <*> time: <*>
--------------------
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2686100
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2688880
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2576380
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1583 time: 0.1959479
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1910 time: 0.2753000
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2590120
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1575 time: 0.1929309
10.11.21.135,10.11.10.1 "GET /openstack/2013-10-17/vendor_data.json H

## evaluate

In [4]:
import time
from utils.evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
m,n,p,q = [],[],[],[]

for dataset in datasets:
    file = f'outputs/parser/0125_0shot_refined/{dataset}.csv'  # Fifth_=_0.1
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a,b,c,d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print('avg---------: group Accuracy: %.4f, Message-Level Accuracy: %.4f, Edit Distance: %.4f' % (sum(m)/len(m), sum(n)/len(n), sum(p)/len(p)))

        HDFS: group Accuracy: 1.0000, Message-Level Accuracy: 0.9425, Edit Distance: 0.0575
      Hadoop: group Accuracy: 0.9815, Message-Level Accuracy: 0.6290, Edit Distance: 7.8830
       Spark: group Accuracy: 0.9230, Message-Level Accuracy: 0.9220, Edit Distance: 1.9545
   Zookeeper: group Accuracy: 0.9930, Message-Level Accuracy: 0.6425, Edit Distance: 1.8015
         BGL: group Accuracy: 0.9645, Message-Level Accuracy: 0.9135, Edit Distance: 2.2955
         HPC: group Accuracy: 0.9145, Message-Level Accuracy: 0.8600, Edit Distance: 1.6260
 Thunderbird: group Accuracy: 0.9805, Message-Level Accuracy: 0.9135, Edit Distance: 0.7615
     Windows: group Accuracy: 0.9960, Message-Level Accuracy: 0.9635, Edit Distance: 0.6300
       Linux: group Accuracy: 0.6845, Message-Level Accuracy: 0.6375, Edit Distance: 4.8695
     Android: group Accuracy: 0.7945, Message-Level Accuracy: 0.6360, Edit Distance: 7.8080
   HealthApp: group Accuracy: 0.7395, Message-Level Accuracy: 0.5255, Edit Dista

## caculate the information entropy

In [ ]:
import math
from collections import Counter
import pandas as pd
import re

def extract_variables(log, template):
    # 将模板中的 <*> 替换为正则表达式的捕获组 (.*?)
    # 为了避免正则表达式的特殊字符导致的问题，先将模板中除了 <*> 外的其他部分进行转义
    # 然后将 <*> 替换为正则表达式的捕获组
    # 这里假设模板中的 <*> 不紧邻正则特殊字符，如果有，需要更复杂的处理
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  # 添加开始和结束锚点以确保完整匹配

    matches = re.search(regex, log)
    if matches:
        return matches.groups()
    else:
        return []

def calculate_entropy(lst):
    # 计算列表中每个元素出现的频率

    # list to str
    # print(''.join(lst))

    counter = Counter(lst)
    probs = [count / len(lst) for count in counter.values()]

    # 计算信息熵
    entropy = -sum(p * math.log2(p) for p in probs)

    return entropy
def select_log_template_pairs_based_on_entropy(pairs, num_examples):
    # 计算每个对的信息熵
    entropies = [(pair, calculate_entropy(list(pair[0]) + list(pair[1])))  # list(pair[0]) + list(pair[1]) / extract_variables(pair[0], pair[1])
                 for pair in pairs]

    # 根据信息熵对对进行排序
    sorted_pairs = sorted(entropies, key=lambda x: x[1], reverse=True)

    # 选择信息熵最高的对
    selected_pairs = sorted_pairs[:num_examples]

    return [pair for pair, entropy in selected_pairs]

# discard the target dataset
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
            'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# datasets.remove('BGL')
demonstration_templates = []
demonstration_logs = []
pairs = []
for d in datasets:
    df = pd.read_csv(f'dataset\{d}\{d}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template in zip(list1, list2):
        if template not in demonstration_templates:
            pairs.append((log, template))
            demonstration_templates.append(template)
            demonstration_logs.append(log)

list =  select_log_template_pairs_based_on_entropy(pairs, 10)
for log, template in list:
    print(log)
    print(template)
    

## Find similarity in all datasets

In [18]:
import pandas as pd
from utils.cluster import tokenize
from utils.sample_byword import extract_variables


datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
    'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']

count_logs = []
count_templates = []

for dataset in datasets:
    print(f"Processing {dataset} ----------------")
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log, template in zip(logs, templates):
        if template not in count_templates:
            count_templates.append(template)
            if any(char.isdigit() for char in template):
                print(f"{template}")

Processing BGL ----------------
rts: bad message header: expecting type <*> instead of type <*> (softheader=<*> <*> <*> <*>) PSR0=<*> PSR1=<*> PRXF=<*> PIXF=<*>
Error receiving packet on tree network, expecting type <*> instead of type <*> (softheader=<*> <*> <*> <*>) PSR0=<*> PSR1=<*> PRXF=<*> PIXF=<*>
<*> L3 EDRAM error(s) (dcr <*>) detected and corrected
dbcr0=<*> dbsr=<*> ccr0=<*>
r24=<*> r25=<*> r26=<*> r27=<*>
critical input interrupt (unit=<*> bit=<*>): warning for tree C1 wire, suppressing further interrupts of same type
size of scratchpad portion of L3.........<*> (<*>)
<*> L3 EDRAM error(s) (dcr <*>) detected and corrected over <*> seconds
fpr29=<*>
Processing HDFS ----------------
Processing Linux ----------------
Processing HealthApp ----------------
flush2DB result success
Processing OpenStack ----------------
Processing OpenSSH ----------------
Failed password for invalid user <*> from <*> port <*> ssh2
Failed password for <*> from <*> port <*> ssh2
Failed none for invali